## MULTI-LABEL IMAGE CLASSIFICATION WITH KERAS

### LOADING LIBRARIES

In [4]:
!pip install pillow
!pip install sklearn

    100% |████████████████████████████████| 2.0MB 674kB/s eta 0:00:01
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import PIL
import PIL.Image

from sklearn.utils import shuffle
import pandas as pd

In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:

import glob,matplotlib as plt
%matplotlib inline

In [7]:
#from keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions

In [4]:
from keras.applications.xception import Xception,preprocess_input, decode_predictions

### LOADING DATA

In [5]:
train=pd.read_csv('./meta-data/train.csv')
test=pd.read_csv('./meta-data/test.csv')
sam=pd.read_csv('./meta-data/sample_submission.csv')

In [6]:
img_width, img_height = 224, 224

In [7]:
train.shape,test.shape

((12600, 86), (5400, 1))

In [8]:

import cv2
def read_image(img_path,H,W):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W,H)) # you can resize to  (128,128) or (256,256)
    return img


In [9]:
import gc
gc.collect()

11

In [ ]:
%%time
image_lr=[]
train_img=[]
for i in list(train.Image_name):
    image_lr.append(read_image('./train_img/'+str(i),224,224))
train_img=np.array(image_lr,dtype=np.float32)
train_img=preprocess_input(train_img)

In [16]:
train_img.shape#,test_img.shape

(12600, 224, 224, 3)

In [13]:

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
#rescale = 1./255,   
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)

'''
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)

'''

gc.collect()


82950

In [18]:
Y=train.iloc[:,1:]
Y.shape

(12600, 85)

In [19]:
'''
l=int(train.shape[0]-(0.1*train.shape[0]))
l
x_train=train_img[:l,:,:,:]
y_train=Y[:l]
x_valid=train_img[l:,:,:,:]
y_valid=Y[l:]
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape


'''

((11340, 224, 224, 3), (11340, 85), (1260, 224, 224, 3), (1260, 85))

In [14]:
batch_size=256

In [41]:


train_generator = train_datagen.flow(train_img,Y,
                                    batch_size = batch_size,
                                    shuffle=True)
                                    #class_mode = "categorical")
'''
validation_generator = test_datagen.flow(x_valid,y_valid,
                                         batch_size = batch_size,
                                        shuffle=True)
                                        #class_mode = "categorical")
'''

In [42]:
model = applications.Xception(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [43]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

In [44]:


#Adding custom Layers 
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(85, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [45]:
adam = Adam(lr=1e-2)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
#validation_data=validation_generator, validation_steps=x_valid.shape[0]//batch_size, 
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=3,shuffle=True)

Epoch 1/3
49/49 [==============================] - 144s 3s/step - loss: 0.3633 - acc: 0.8348
Epoch 2/3
49/49 [==============================] - 141s 3s/step - loss: 0.2396 - acc: 0.8960
Epoch 3/3
49/49 [==============================] - 140s 3s/step - loss: 0.2111 - acc: 0.9094


In [49]:
# Train the model 
k.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,shuffle=True)
#model_final.fit(train_img, Y,batch_size=256,epochs=3,shuffle=True,verbose=1)

Epoch 1/2
49/49 [==============================] - 144s 3s/step - loss: 0.1886 - acc: 0.9201
Epoch 2/2
49/49 [==============================] - 141s 3s/step - loss: 0.1821 - acc: 0.9233


In [50]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model_final.layers[:126]:
    layer.trainable = False
for layer in model_final.layers[126:]:
    layer.trainable = True

In [51]:
adam = Adam(lr=1e-4)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=1, shuffle=True)

Epoch 1/1
49/49 [==============================] - 145s 3s/step - loss: 0.1742 - acc: 0.9266


In [53]:
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,shuffle=True)

Epoch 1/2
49/49 [==============================] - 147s 3s/step - loss: 0.1624 - acc: 0.9316
Epoch 2/2
49/49 [==============================] - 141s 3s/step - loss: 0.1574 - acc: 0.9337


In [54]:
# serialize model to JSON
from keras.models import model_from_json
model_json = model_final.to_json()
with open("dl3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("dl3.h5")
print("Saved model to disk")
 
# later...


Saved model to disk


### PREDICTIONG / SUBMISSION

##### LOAD MODEL AND WEIGHTS

In [10]:
gc.collect()

0

In [11]:
%%time
image_lr=[]
test_img=[]
for i in list(test.Image_name):
    image_lr.append(read_image('./test_img/'+str(i),224,224))
test_img=np.array(image_lr,dtype=np.float32)
test_img=preprocess_input(test_img)

CPU times: user 1min 30s, sys: 1.52 s, total: 1min 32s
Wall time: 1min 32s


In [12]:
from keras.models import model_from_json
# load json and create model
json_file = open('dl3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("dl3.h5")
print("Loaded model from disk")



Loaded model from disk


In [15]:
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)

batch_size=256

In [19]:
test_generator = test_datagen.flow(test_img,batch_size = batch_size,shuffle=True)                                      
pred=model_final.predict_generator(test_generator,verbose=1)


22/22 [==============================] - 71s 3s/step


In [21]:
test.shape,test_img.shape

((5400, 1), (5400, 224, 224, 3))

In [22]:
pred.shape

(5400, 85)

In [31]:

sam['Image_name']=test.Image_name
sam.iloc[:,1:]=pred
sam.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0.117319,1.659853e-05,8.829146e-08,0.028858,7.984019e-04,0.999267,0.020317,2.430703e-03,0.948949,...,0.232922,0.000869,0.968741,0.780258,9.820098e-01,0.004417,9.021670e-10,0.991577,4.473517e-06,0.916900
1,Image-2.jpg,1.000000,4.705302e-09,9.999999e-01,1.000000,7.335226e-13,0.000004,0.000007,2.117965e-07,0.996960,...,1.000000,0.975226,0.999997,0.000019,2.459377e-06,0.999972,5.039305e-14,0.999979,5.428850e-10,0.000077
2,Image-3.jpg,0.999810,2.015840e-08,1.013792e-05,0.999996,2.509124e-14,0.000199,0.000048,9.999944e-01,0.999974,...,0.999329,0.000004,0.000099,0.999991,8.449949e-07,0.999958,2.636226e-16,0.999995,5.494431e-07,0.919105
3,Image-4.jpg,0.882191,1.208933e-02,1.149171e-02,0.358601,2.647758e-01,0.816245,0.235523,2.062053e-02,0.117917,...,0.586667,0.418045,0.859593,0.699928,1.341764e-01,0.030996,4.071902e-04,0.802190,1.365234e-02,0.282143
4,Image-5.jpg,0.151068,4.547408e-05,3.547970e-07,0.053564,8.809268e-04,0.998528,0.023123,3.654342e-03,0.943171,...,0.277097,0.002486,0.979284,0.819130,9.697222e-01,0.008256,3.249107e-09,0.986364,1.381546e-05,0.865403


In [2]:
sam=pd.read_csv('sub-xception.csv')
sam.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0.117319,1.659853e-05,8.829146e-08,0.028858,7.984019e-04,0.999267,0.020317,2.430703e-03,0.948949,...,0.232922,0.000869,0.968741,0.780258,9.820098e-01,0.004417,9.021670e-10,0.991577,4.473517e-06,0.916900
1,Image-2.jpg,1.000000,4.705302e-09,9.999999e-01,1.000000,7.335226e-13,0.000004,0.000007,2.117965e-07,0.996960,...,1.000000,0.975226,0.999997,0.000019,2.459377e-06,0.999972,5.039305e-14,0.999979,5.428850e-10,0.000077
2,Image-3.jpg,0.999810,2.015840e-08,1.013792e-05,0.999996,2.509124e-14,0.000199,0.000048,9.999944e-01,0.999974,...,0.999329,0.000004,0.000099,0.999991,8.449949e-07,0.999958,2.636226e-16,0.999995,5.494431e-07,0.919105
3,Image-4.jpg,0.882191,1.208933e-02,1.149171e-02,0.358601,2.647758e-01,0.816245,0.235523,2.062053e-02,0.117917,...,0.586667,0.418045,0.859593,0.699928,1.341764e-01,0.030996,4.071902e-04,0.802190,1.365234e-02,0.282143
4,Image-5.jpg,0.151068,4.547408e-05,3.547970e-07,0.053564,8.809268e-04,0.998528,0.023123,3.654342e-03,0.943171,...,0.277097,0.002486,0.979284,0.819130,9.697222e-01,0.008256,3.249107e-09,0.986364,1.381546e-05,0.865403


In [14]:
temp=sam.copy()
for i in sam.iloc[:,1:]:
    temp[i]=sam[i].apply(lambda x: 1 if x>0.00005 else 0)
        

In [15]:
temp.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,1,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,0,1
1,Image-2.jpg,1,0,1,1,0,0,0,0,1,...,1,1,1,0,0,1,0,1,0,1
2,Image-3.jpg,1,0,0,1,0,1,0,1,1,...,1,0,1,1,0,1,0,1,0,1
3,Image-4.jpg,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Image-5.jpg,1,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,0,1


In [16]:

temp.to_csv('sub-xception-1.csv',index=False)
temp.shape

(5400, 86)